# ConvNet 시각화
- 컨브넷 중간층의 출력(중간층에 있는 활성화) 시각화
  - 입력된 컨브넷 층이 입력을 어떻게 변형시키는 이해. 개별적인 컨브넷 필터의 의미 파악
- 클래스 활성화에 대한 히트맵을 이미지에 시각화
  - 이미지의 어느 부분이 주어진 클래스에 속하는데 기여했는지 이해
  - 이미지에서 객체 위치를 추정하는데 도움이 됨

In [ ]:
import tensorflow as tf
tf.__version__


## **1. 중간층 활성화 시각화**

- 어떤 입력이 주어졌을 때 네트워크에 있는 여러 합성곱과 풀링 층이 출력하는 특성 맵을 그리는 것
- 층의 출력이 활성화 함수의 출력이므로 활성화라 부름
- 네트워크에 의해 학습된 필터들이 어떻게 입력을 분해하는지 보여줌
- 너비, 높이, 깊이(채널) 3개의 차원에 대한 특성 맵 시각화
  - 각 채널은 비교적 독립적인 특성을 인코딩하므로 특성 맵의 각 채널 내용을 독립적인 2D 이미지로 그림

**1) 네트워크 모델 로드**
- Dogs and Cats small 데이터 증식하여 생성한 네트워크 모델 로드

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('./model/cats_and_dogs_small_augmentation.h5')
model.summary()

**2) 개별 이미지 전처리**
- 새로운 고양이 이미지를 입력 이미지로 선택 

In [ ]:
img_path = './datasets/cats_and_dogs_small/test/cats/cat.1700.jpg'

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img = load_img(img_path, target_size=(150,150))
img_tensor = img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)   # 이미지를 4D텐서로 변환
img_tensor /= 255.

print(img_tensor.shape)


- 선택 이미지 출력

In [ ]:
 import matplotlib.pyplot as plt

 plt.imshow(img_tensor[0])
 plt.show()

**3) 입력 텐서와 출력 텐서의 리스트로 모델 객체 생성**
- keras의 Model 클래스를 사용하여 모델 생성
  - 모델 객체 생성을 위한 2개의 매개변수 필요
    - 입력 텐서(또는 입력 텐서 리스트)
    - 출력 텐서(또는 출력 텐서 리스트)
  - 반환 객체
    - 특정 입력과 특정 출력을 매핑한 Sequential과 같은 케라스 모델

In [ ]:
from tensorflow.keras.models import Model

# 상위 8개 층의 출력을 추출
layer_outputs = [layer.output for layer in model.layers[:8]]

# 입력에 대해 8개 층의 출력을 반환하는 모델 생성
activation_model = Model(inputs=model.input, outputs=layer_outputs)

**4) 예측 모드로 모델 실행(predict)**

In [ ]:
activations = activation_model.predict(img_tensor)

**5) 중간층 활성화 시각화**
- 첫번째 합성곱 층 활성화

In [ ]:
# 고양이 이미지에 대한 첫번째 합성곱 층 활성화
first_layer_activation = activations[0]
print(first_layer_activation.shape)

- 첫번째 층 활성화 중 20번째 채널 시각화

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0,:,:,19], cmap='viridis')

- 첫번째 층 활성화 중 16번째 채널 시각화

In [ ]:
plt.matshow(first_layer_activation[0,:,:,15], cmap='viridis')

- 네트워크의 모든 중간층(8개) 활성화 시각화

In [ ]:
layer_names = []
for layer in model.layers[:8]:
  layer_names.append(layer.name)

images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations):
  n_features = layer_activation.shape[-1]     # 특성 맵에 있는 특성의 수
  size = layer_activation.shape[1]            # 특성 맵의 크기는 (1, size, size, n_features)
  n_cols = n_features // images_per_row       # 활성화 채널을 위한 그리드 크기 계산
  display_grid = np.zeros((size*n_cols, images_per_row*size))

  # 각 활성화를 하나의 큰 그리드에 채움
  for col in range(n_cols):
    for row in range(images_per_row):
      channel_image = layer_activation[0, :, :, col*images_per_row+row]
      
      # 그래프로 나타내기 좋게 특성 변환
      channel_image -= channel_image.mean()
      channel_image /= channel_image.std()
      channel_image *= 64
      channel_image += 128

      channel_image = np.clip(channel_image, 0, 255).astype('uint8')

      # 그리드 출력
      display_grid[col*size : (col+1)*size, row*size:(row+1)*size] = channel_image

  scale = 1./size
  plt.figure(figsize=(scale*display_grid.shape[1],
                    scale*display_grid.shape[0])) 
  plt.title(layer_name)
  plt.grid(False)
  plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()



---



---



## **2. 클래스 활성화의 히트맵 시각화**

**클래스 활성화 맵(class activation map: CAM)**
- 특정 출력 클래스에 대해 입력 이미지의 모든 위치를 계산한 2D점수 그리드
- 이미지의 어느 부분이 컨브넷의 최종 분류 결정에 기여하는지 이해하는데 유용함
- 분류에 실수가 있는 경우 컨브넷의 결정 과정을 디버깅하는데 도움이 됨
- 이미지에 특정 물체가 있는 위치를 파악하는데 사용할 수 있음

**클래스 활성화 맵 시각화 기술**
- Grad-CAM: Visual Explanations from Deep Networkd via Gradient-based Localization
  - 입력 이미지가 주어지면 합성곱 층에 있는 특성 맵의 출력을 추출
  - 특성 맵의 모든 채널 출력에 채널에 대한 클래스의 그래디언트 평균을 곱함

- '입력 이미지가 각 채널을 활성화하는 정도'에 대한 공간적인 맵을 '클래스에 대한 각 채널의 중요도'로 가중치를 부여하여 '입력 이미지가 클래스를 활성화하는 정도'에 대한 공간적인 맵을 만드는 것

예. VGG16 네트워크를 이용한 클래스 활성화 맵

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet')

VGG16을 위한 입력 이미지 전처리
- 샘플이미지 : 초원을 걷는 어미와 새끼 코끼리 이미지
- 이미지 로드 후 224*224 크기로 변경하고 넘파이 float32 텐서로 변경 후 전처리 함수 적용

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img_path = './datasets/creative_commons_elephant.jpg'

img = load_img(img_path, target_size= (224,224))
x = img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

In [ ]:
sample_img = plt.imread('./datasets/creative_commons_elephant.jpg')
plt.imshow(sample_img)
plt.show()

샘플 이미지에 대한 사전 훈련된 네트워크 실행
- 예측 벡터 : pred

In [ ]:
pred = model.predict(x)

디코딩

In [ ]:
print('Predicted:', decode_predictions(pred, top=3)[0])

예측 벡터에서 최대로 활성화된 항목

In [ ]:
np.argmax(pred[0])

**Grad-CAM 알고리즘 설정**

In [ ]:
from keras import backend as K
african_elephant_output = model.output[:, 386]

last_conv_layer = model.get_layer('block5_conv3')
grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))
print(pooled_grads)
iterate = K.function([model.input],
                     [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
  conv_layer_output_value[:,:,i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

**히트맵 후처리**
- 시각화를 위해 0~1사이로 정규화

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

plt.matshow(heatmap)

**openCV를 사용하여 원본 이미지에 히트맵 붙이기**

In [ ]:
import cv2

img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255*heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('./datasets/elephant_cam.jpg', superimposed_img)


원본이미지에 클래스 활성화 히트맵을 겹친 이미지

In [ ]:
result = plt.imread('./datasets/elephant_cam.jpg')

plt.imshow(result)
plt.show